In [ ]:
# !pip install pandas

In [ ]:
import sys
print(sys.executable) 

In [ ]:
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install pandas
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install icecream
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install nltk
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install plotly
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install pytorch_pretrained_bert pytorch-nlp
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install keras
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install tensorflow
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install transformers
! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install ipynb

In [1]:
import pandas as pd
import numpy as np
import random as rn
from tqdm.notebook import tqdm
from icecream import ic 

import nltk
# from nltk.tokenize import regexp_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# imports from shudima
# import torch
# from pytorch_pretrained_bert import BertModel
# from torch import nn
# from torchnlp.datasets import imdb_dataset
# from pytorch_pretrained_bert import BertTokenizer

# imports from colbert
# import tensorflow as tf
# import bert_tokenization as tokenization
# import tensorflow.keras.backend as K
# from tensorflow import keras
# from transformers import *
from transformers import BertTokenizer

# imports from shudima
# from keras.preprocessing.sequence import pad_sequences
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from torch.optim import Adam
# from torch.nn.utils import clip_grad_norm_
# from IPython.display import clear_output

# import re
# import os

In [2]:
#init 
train_count = 1000 #8000
# test_count = 

MAX_SENTENCE_LENGTH = 128#colbert=20
MAX_SENTENCES = 5
MAX_LENGTH = 100

In [ ]:
train_df = pd.read_csv("data/train.csv")
# display(list(train_df))
display(train_df.count())

In [ ]:
train = pd.read_csv("data/train.csv", skipinitialspace=True, usecols=['id','text','is_humor'])    #read first 3 columns
# test = pd.read_csv(test_loc)
train = train[:train_count]

# preprocessing section
# lower
train['text'] = train['text'].str.lower()
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
# special char
for c in spec_chars:
    train['text'] = train['text'].str.replace(c, ' ', regex=True)
    
#replce double space
# train['text'] = train['text'].str.replace('  ', ' ')
# train['text'] = re.sub(r"</?\[\d+>", "", train['text'])  
train['text'] = train['text'].str.replace('\s+', ' ', regex=True) 

# print
display(train)
display(train.dtypes)

# TOKENIZE
train['text'] = train['text'].astype(str)
# file_dev['Correct Statement']
train['word_tokens'] = train.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
# display(train)
# file_dev=file_dev.drop(['Correct Statement'], axis = 1)

# LEMMA
stemmer = PorterStemmer()

def get_stem_list(row):
    stemmed_list = [stemmer.stem(word) for word in row]
    return (stemmed_list)

train['stem_word_tokens'] = train['word_tokens'].apply(get_stem_list)
# display(train['stem_word_tokens'])

# STOP WORDS
stops = set(stopwords.words("english"))                  

def delete_stops(row):
    meaningful_words = [w for w in row if not w in stops]
    return (meaningful_words)

train['del_stop_word_tokens'] = train['stem_word_tokens'].apply(delete_stops)
# train['del_stop_word_tokens'] = train['word_tokens'].apply(delete_stops)

display(train)

# DROP
# train = train['word_tokens'].drop()
# train = train['stem_word_tokens'].drop()
# file_dev.to_csv('file_dev_processed.csv', index=False)


In [ ]:
# nltk.download('punkt')

In [ ]:
# nltk.download('stopwords')

In [ ]:
# plot frequent words for treds.

from collections import Counter
import plotly.graph_objects as go
import plotly.offline as pyo

n=1000

# without lemma, and no stop words
# sp = train['word_tokens'].apply(delete_stops)
# with lemma, no stop words
sp = train['del_stop_word_tokens']
sim = sp.values.tolist()
merged_list = []
for l in sim:
    merged_list += l
freq = Counter(merged_list).most_common(n)
# display(freq)

# x_axis = list(range(1, n+1)) #rank of words
x_axis = list(dict(freq).keys())
y_axis = list(dict(freq).values())

pyo.init_notebook_mode()

fig = go.Figure(data = go.Scatter(x= x_axis, y= y_axis))
fig.update_layout(title='most frequent words',
                   xaxis_title='words',
                   yaxis_title='Frequency')
# fig.show()
pyo.iplot(fig, filename = 'basic-line')


In [ ]:
# preprocessing for BERT

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer.tokenize('Hi my nambe is Dima')
# train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train['text']))

In [17]:
from ipynb.fs.full.utils import *
from ipynb.fs.full.datasets import *
from ipynb.fs.full.architectures import *
from ipynb.fs.full.train_test import *

In [18]:
# sys.path.append(".")
batch_size = 10 #32 or 128


traindata = torch.utils.data.DataLoader(dataset= Hahackathon("data/train.csv", basenet= 'bert'), batch_size= batch_size, shuffle= True)
# ic("Size of the dataset",len(traindata.dataset))
# model = multitask_lstm_fc('bert') #multitask lstm fc is for multitask not single task
model = multitask_lstm_fc('bert')

Sampled input from the file: data/train.csv
   id                                               text  is_humor  \
0   1  TENNESSEE: We're the best state. Nobody even c...         1   
1   2  A man inserted an advertisement in the classif...         1   
2   3  How many men does it take to open a can of bee...         1   
3   4  Told my mom I hit 1200 Twitter followers. She ...         1   
4   5  Roses are dead. Love is fake. Weddings are bas...         1   

   humor_rating  humor_controversy  offense_rating  
0          2.42                1.0             0.2  
1          2.50                1.0             1.1  
2          1.95                0.0             2.4  
3          2.11                1.0             0.0  
4          2.78                0.0             0.1  
Tokenizer: bert-base-uncased

Base architecture: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/Users/kshitijahande/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all 

In [19]:
# parser.add_argument('--lr_max',            type = float,          default = 0.00002,      help = 'learning rate (default: 0.01)')
learning_rate=0.002 #0.00002
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)


In [20]:
# ic.enable()
from sklearn.metrics import roc_auc_score, mean_squared_error, f1_score, accuracy_score
 
# mse_criterion = torch.nn.MSELoss()
ce_criterion  = torch.nn.CrossEntropyLoss()
ic(ce_criterion)
loss_arr=[]

def concat_output1(output, device):
    temp = torch.tensor([]).to(device)

    if output.shape[1] == 2:
        ic()
        temp = torch.argmax(output[:, 0:2], dim=1).view(-1,1)
#         ic(temp)
        return temp

    temp = torch.argmax(output[:, 0:2], dim=1).view(-1,1)
    temp = torch.cat((temp, torch.argmax(output[:, 2:4], dim= 1).view(-1,1)), dim= 1)
    temp = torch.cat((temp, output[:, 4].view(-1,1)), dim= 1)
    temp = torch.cat((temp, output[:, 5].view(-1,1)), dim= 1)
    return temp

def loss_func1(output, target, device):

    humor_id = (target[:, 0]==1)
    target   = target.float()
#     ic(output[:, 0:2])
    
    loss1 = ce_criterion(output[:, 0:2], target[:, 0].long())	                	# is_humor binary classification
#     loss2 = ce_criterion(output[humor_id][:, 2:4], target[humor_id][:, 1].long())	# humor_controversy binary classification
#     loss3 = mse_criterion(output[humor_id][:, 4], target[humor_id][:, 2])			# humor rating regression loss
#     loss4 = mse_criterion(output[:, 5], target[:, 3])								# offensive rating regression loss
#     loss  = loss1+loss2+loss3+loss4
    loss=loss1
    
    # this block of generating temp is verified throughly
    temp = concat_output1(output, device)
#     ic('outside concat_output ', temp)
    temp = temp.detach()

    return loss, temp


def train1(model, dataloader, optimizer, device):
#     ic('1')
    model.train()
#     ic('2')
    train_loss = AverageMeter()

    for batch_id, batch_data in enumerate(dataloader):
#         ic(batch_data[0], batch_data[1])
        data       = batch_data[0]
        target     = batch_data[1].to(device)
#         ic(data)
#         ic(target)

        input_id   = data[0]
        mask_id    = data[1]
        segment_id = data[2]
#         ic(input_id)
#         ic(mask_id)
#         ic(segment_id)
        
        output = model(input_id, mask_id, segment_id)
#         ic('output model')
        
        loss, _ = loss_func1(output, target, device= device)
        ic('outside loss_func')
        loss_arr.append(loss)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.update(loss.item(), target.shape[0])

    return train_loss.avg

ic| ce_criterion: CrossEntropyLoss()


In [21]:
# %%time

use_cuda= False
device   = torch.device("cuda" if use_cuda else "cpu")
model = model.to(device)

# ic(traindata)
# ic(optimizer)

train_loss = train1(model, traindata, optimizer, device)

AttributeError: 'list' object has no attribute 'size'

In [ ]:
ic(train_loss)

In [ ]:
# # from IITK code
# # note - DO NOT USE STOP WORDS
# # function from datasets.py
# def get_tokenized_text(self, text):
#     # marked_text = "[CLS] " + text + " [SEP]"
# #     max_length= 128, stop_words= False, basenet=bert, return_tensors= nope
#     train_encoded_inputs = tokenizer(text= text, # the sentence to be encoded
#                     add_special_tokens= True,  # add [CLS] and [SEP]
#                     max_length= MAX_SENTENCE_LENGTH  # maximum length of a sentence
#                     padding= 'max_length',  # add [PAD]s
#                     return_attention_mask= True,  # generate the attention mask
# #                     return_tensors = 'pt',  # return PyTorch tensors
#                     truncation= True) 

#     input_id = encoded['input_ids']
#     mask_id  = encoded['attention_mask']
# # input_segments= something related to tensors??? - it is sentence id to distinguish between sentences! - IMP 
# # , label= id, text, is _humor ??
#     segment_id = [1] * MAX_SENTENCE_LENGTH

#     return input_id, mask_id, segment_id


# def compute_input_arrays(df, columns, tokenizer):
#     model_input = []
# #     for xx in range((MAX_SENTENCES*3)+3):
# #         model_input.append([])
    
#     for _, row in tqdm(df[columns].iterrows()):
#         i = 0
        
#         # sent
#     ic(model_input[0].shape)
#     return model_input

# # train_encoded_inputs = compute_input_arrays(train, list(train.columns[[1]]), tokenizer)

In [ ]:
# # from colbert code
# def return_id(str1, str2, truncation_strategy, length):

# #     ic(str1)
# #     ic(str2)
#     inputs = tokenizer.encode_plus(str1, str2,
#         add_special_tokens=True,
#         max_length=length,
#         truncation_strategy=truncation_strategy)

# #     ic(inputs)
# #     ic(inputs["input_ids"])
#     input_ids =  inputs["input_ids"]
#     input_masks = [1] * len(input_ids)
#     input_segments = inputs["token_type_ids"]
#     padding_length = length - len(input_ids)
#     padding_id = tokenizer.pad_token_id
#     input_ids = input_ids + ([padding_id] * padding_length)
#     input_masks = input_masks + ([0] * padding_length)
#     input_segments = input_segments + ([0] * padding_length)

#     return [input_ids, input_masks, input_segments]


# def compute_input_arrays(df, columns, tokenizer):
#     model_input = []
#     for xx in range((MAX_SENTENCES*3)+3):
#         model_input.append([])
    
#     for _, row in tqdm(df[columns].iterrows()):
#         i = 0
        
#         # sent
# #         ic(row)
# #         ic(row.text)
#         sentences = sent_tokenize(row.text)
#         for xx in range(MAX_SENTENCES):
#             s = sentences[xx] if xx<len(sentences) else ''
# #             ic(xx)
#             ids_q, masks_q, segments_q = return_id(s, None, 'longest_first', MAX_SENTENCE_LENGTH)
#             model_input[i].append(ids_q)
#             i+=1
#             model_input[i].append(masks_q)
#             i+=1
#             model_input[i].append(segments_q)
#             i+=1
        
#         # full row
#         ids_q, masks_q, segments_q = return_id(row.text, None, 'longest_first', MAX_LENGTH)
#         model_input[i].append(ids_q)
#         i+=1
#         model_input[i].append(masks_q)
#         i+=1
#         model_input[i].append(segments_q)
        
#     for xx in range((MAX_SENTENCES*3)+3):
#         model_input[xx] = np.asarray(model_input[xx], dtype=np.int32)
#     ic('model input is train examples x bert features')
#     ic(model_input[0].shape)
#     return model_input

In [ ]:
# ic(list(train.columns[[1]]))
# # train_encoded_inputs
# train_encoded_inputs = compute_input_arrays(train, list(train.columns[[1]]), tokenizer)


In [ ]:
# ic('model inputs, rows, colums/bert features')
# ic(len(train_encoded_inputs), len(train_encoded_inputs[0]), len(train_encoded_inputs[0][0]))

# # check out input for 7th row
# r = 7
# ic(train.iloc[r])
# ic(train.iloc[r,1])
# ic(sent_tokenize(train.iloc[r,1]))
# # en_text[0][xx], en_text[3][xx], en_text[6][xx], en_text[15][xx]
# train_encoded_inputs[0][r], train_encoded_inputs[3][r], train_encoded_inputs[6][r], train_encoded_inputs[15][r]
